In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.8 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/avisinghal6/VIDEO_CAPTIONING_PIPELINE.git

Cloning into 'VIDEO_CAPTIONING_PIPELINE'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 45 (delta 20), reused 24 (delta 8), pack-reused 0
Unpacking objects: 100% (45/45), 8.52 MiB | 6.43 MiB/s, done.


In [3]:
%cd VIDEO_CAPTIONING_PIPELINE

/content/VIDEO_CAPTIONING_PIPELINE


In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
from frames import *
from model import *
from captions import *
from yolo import *
import cv2

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.9/dist-packages/transformers/models/yolos/image_processing_yolos.py:710: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(


In [5]:
#GENERATING FRAMES
videos_list={}
videos_time_stamps=get_saving_frames_durations(5,"dvc_results.json")
for time_stamp in videos_time_stamps:
  for key,value in time_stamp.items():
    generate_frames(f'{key}.mp4',value,videos_list)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
#GENERATING FEATURES
img2vec = Img2VecResnet18(device) 
videos_features_vectors={}
for time_stamp in videos_time_stamps:
  allVectors={}
  for key,value in time_stamp.items():
    for image in tqdm(os.listdir(f'{key}-opencv')):
      I = Image.open(os.path.join(f'{key}-opencv', image))
      vec = img2vec.getVec(I)
      allVectors[image] = vec
      I.close() 
    videos_features_vectors[key]=allVectors

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 200MB/s]
100%|██████████| 351/351 [00:16<00:00, 21.06it/s]


In [9]:
#COMPUTING SIMILARITY BETWEEN FRAMES
from collections import defaultdict
result=get_similar_frames(videos_features_vectors,videos_list)

In [10]:
#GETTING FINAL FRAMES FOR EACH VIDEO
final_frames=get_final_frames(result)

In [8]:
#USE FINAL FRAMES FOR CAPTIONING

In [12]:
#USING YOLO FOR OBJECT DETECTION AND ADDING THE BOUNDING BOXES ALONG WITH LABELS
for k,v in final_frames.items():
  for k1,v1 in v.items():
    yolo_box(k,v1)


In [13]:
#GET CAPTIONS
config={
    "processor": "Salesforce/blip-image-captioning-base",

    "model":"Salesforce/blip-image-captioning-base"
}
processor, model = get_model(config,device)
captions = get_captions(final_frames, processor, model,device)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [16]:
captions


{'---AfxeTnCbVQ': 'a person is stirring blue food into a pan. a blue and green colored substance in a metal bowl. a blue spat is being used to make blue food. a pan of blue food that is being cooked. blue and black colored pigments. a col of photos showing how to make a torta. a blue and white roll with a picture of a blue bowl. a stuffed animal is laying on the floor. ',
 '----bv0V6ZjWI': 'a blue and white substance with the words 2 0 red onion. a person is putting blue and green rocks into a black pan. a person is stirring blue glass beads into a pan. a blue bowl with a black substance inside. blue slim slim slim slim slim slim slim slim slim slim slim slim slim slim slim slim slim slim. a col of photos showing how to make a torta. melted pepper cheese dip dip. someone is peeling a piece of a white egg. '}

In [15]:
#ADD CODE FOR LLM